# GPT-4 API Interaction with Name Replacement
This notebook demonstrates how to connect to the GPT-4 API using OpenAI's Python client, modify a prompt by replacing a name, send the modified prompt to the API, and then replace the name in the response.

In [ ]:
!pip install openai==0.28.0

In [ ]:
# -----------------------------------------------------------
# This script generates two modified prompts by substituting the name
# "Akecheta" with a randomly selected name from a predefined list
# and a fixed name "User123". It sends these prompts to the OpenAI GPT-4
# model to generate responses. After retrieving the responses,
# it replaces the substituted names back to "Akecheta" and computes
# the semantic similarity between the two final responses using
# OpenAI's Embedding API and cosine similarity.
#
# Dependencies:
# - openai==0.28.0
# - numpy
#
# Usage:
# 1. Ensure required libraries are installed.
# 2. Run the script and enter your OpenAI API key when prompted.
# 3. The script will display the modified prompts, final responses,
#    and the semantic similarity score with an interpretation.
# -----------------------------------------------------------

# Install the necessary libraries. Uncomment the next lines if not already installed.
# !pip install openai==0.28.0
# !pip install numpy


import openai
import random
import numpy as np
from getpass import getpass

def get_api_key():
    """
    Securely get the OpenAI API key from the user.
    """
    return getpass('Enter your OpenAI API key: ')

def select_random_name(alternative_names):
    """
    Select a random name from the provided list.
    """
    return random.choice(alternative_names)

def send_prompt(model, prompt):
    """
    Send the prompt to the specified OpenAI model and return the assistant's reply.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model,  # Ensure you have access to the specified model
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Adjust as needed
            temperature=0.7   # Adjust for creativity
        )
        return response['choices'][0]['message']['content'].strip()
    except openai.error.OpenAIError as e:
        print(f"An error occurred while communicating with the OpenAI API: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return ""

def get_embedding(text, model="text-embedding-ada-002"):
    """
    Generate an embedding for the given text using OpenAI's Embedding API.
    """
    try:
        response = openai.Embedding.create(input=text, model=model)
        return response['data'][0]['embedding']
    except openai.error.OpenAIError as e:
        print(f"An error occurred while generating embeddings: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

def cosine_similarity(vec1, vec2):
    """
    Compute the cosine similarity between two vectors.
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        print("One of the vectors is zero; cannot compute cosine similarity.")
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def main():
    # Step 1: Securely get the OpenAI API key from the user
    api_key = get_api_key()
    openai.api_key = api_key

    # Step 2: Define the list of alternative names
    alternative_names = ["Ahanu", "Chayton", "Dakota"]

    # Define the original name and prompt
    original_name = "Akecheta"
    prompt_template = f"Prepare a short speech introducing {original_name} in three points, highlighting heritage."

    # ---------------------------------------------
    # First Task: Random Name Substitution
    # ---------------------------------------------
    # Step 3: Select a random name from the list
    selected_name_random = select_random_name(alternative_names)

    # Step 4: Replace "Akecheta" with the randomly selected name
    modified_prompt_random = prompt_template.replace(original_name, selected_name_random)

    print(f"\nModified Prompt with Random Name ({selected_name_random}):\n{modified_prompt_random}\n")

    # Step 5: Send the modified prompt to the GPT-4o API using ChatCompletion
    response_random = send_prompt("gpt-4o", modified_prompt_random)  # Replace "gpt-4" with "gpt-4o" if applicable

    # Step 6: Replace the randomly selected name with "Akecheta" in the response
    final_response_random = response_random.replace(selected_name_random, original_name)

    # Step 7: Display the final response
    print("Final Response with Original Name (Akecheta):\n")
    print(final_response_random)
    print("\n" + "-"*80 + "\n")

    # ---------------------------------------------
    # Second Task: "User123" Name Substitution
    # ---------------------------------------------
    # Step 3: Define the substitution name
    substitution_name_fixed = "User123"

    # Step 4: Replace "Akecheta" with "User123"
    modified_prompt_fixed = prompt_template.replace(original_name, substitution_name_fixed)

    print(f"\nModified Prompt with Fixed Name ({substitution_name_fixed}):\n{modified_prompt_fixed}\n")

    # Step 5: Send the modified prompt to the GPT-4 API using ChatCompletion
    response_fixed = send_prompt("gpt-4", modified_prompt_fixed)  # Replace "gpt-4" with "gpt-4o" if applicable

    # Step 6: Replace "User123" with "Akecheta" in the response
    final_response_fixed = response_fixed.replace(substitution_name_fixed, original_name)

    # Step 7: Display the final response
    print("Final Response with Original Name (Akecheta):\n")
    print(final_response_fixed)
    print("\n" + "-"*80 + "\n")

    # ---------------------------------------------
    # Semantic Similarity Comparison
    # ---------------------------------------------
    print("Comparing the two final responses for semantic similarity...\n")

    # Ensure both responses are non-empty
    if not final_response_random or not final_response_fixed:
        print("One or both responses are empty. Cannot compute similarity.")
        return

    # Step 8: Generate embeddings for both responses
    embedding_random = get_embedding(final_response_random)
    embedding_fixed = get_embedding(final_response_fixed)

    if embedding_random is None or embedding_fixed is None:
        print("Failed to generate embeddings for one or both responses.")
        return

    # Step 9: Compute cosine similarity between the two embeddings
    similarity_score = cosine_similarity(embedding_random, embedding_fixed)

    # Step 10: Display the similarity score
    print(f"Semantic Similarity (Cosine Similarity): {similarity_score:.4f}")

    # Optional: Interpret the similarity score
    if similarity_score >= 0.8:
        interpretation = "The responses are highly similar."
    elif similarity_score >= 0.5:
        interpretation = "The responses are moderately similar."
    else:
        interpretation = "The responses are not very similar."

    print(f"Interpretation: {interpretation}")

if __name__ == "__main__":
    main()
